In [13]:
# Rode para instalar os requirements
! python -u -m pip install -r requirements.txt

In [14]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from arb import Arbitrage
import time
from typing import List

In [15]:
CASAS_DE_APOSTA_BR = [
    'Dafabet',
    'Betfair',
    'Betway',
    'Betano',
    'Bet365',
    'Galera Bet',
    'Betsson',
    '22BET',
    'Betwinner',
    'Rabona',
    '888sport',
    'LeoVegas',
    'Rivalo',
    'Parimatch',
    'Sportaza',
    'Betkwiff',
    'Amuletobet',
    '20BET',
    'Powbet',
    'FezBet',
    'Campeonbet',
    'Melbet',
    'Pixbet',
    'BetWarrior',
    'Bet7',
    'Megapari',
    'KTO',
    'Vbet',
    'CampoBet',
    'CyberBet',
    'LVBET',
    'Betmaster',
    'Bodog',
    'Fansbet',
    'Betmotion'
]

CASAS_DE_APOSTA_BR = list(map(lambda s: s.lower(), CASAS_DE_APOSTA_BR))

available_games = list()


In [16]:
# Baixa as casas de aposta brasileiras para um determinado jogo
def retrieve_bet_houses_info_from_spec_game(parsed_text: List[str]):
    sites = dict()
    inc = 0
    try:
        while True:
            casa_aposta, odd_a, empate, odd_b, payout = parsed_text[5+inc:10+inc]
            casa_aposta = casa_aposta.lower()
            if 'Log in to display the odds!' in [casa_aposta, odd_a, empate, odd_b, payout]:
                break

            if casa_aposta in CASAS_DE_APOSTA_BR:
                sites[casa_aposta] = {
                    "Odd_a":    float(odd_a),
                    "Empate":   float(empate),
                    "Odd_b":    float(odd_b),
                    "Payout":   payout
                }
            inc += 5
    except:
        pass
    
    # print("Sites obtidos: ", sites)
    return pd.DataFrame(sites).T

In [17]:
def retrieve_game_parsed_text(driver, URL: str):
    webElement = None
    
    print("URL atual: {}".format(URL))
    try:
        driver.get(URL)
    except:
        pass

    webElement = None
    while webElement is None:
        try:
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "odds-data-table")))
            webElement = driver.execute_script('return document.getElementById("odds-data-table").outerHTML')
        except TimeoutException:
            pass
    

    title = driver.execute_script('return document.title')
    print(title)
    
    assert isinstance(webElement, str)
    
    SoupElement = BeautifulSoup(webElement, 'html.parser')
    raw_text = [text for text in SoupElement.stripped_strings]
    # print(raw_text)

    return retrieve_bet_houses_info_from_spec_game(raw_text)

In [20]:
url = "https://www.oddsportal.com/soccer/europe/europa-conference-league/"
SUBURL = url[26:]
driver = webdriver.Chrome()
webElement = None

try:
    driver.maximize_window()
    driver.get(url)
    # Login no oddsPortal
    time.sleep(2)
    driver.execute_script('document.getElementsByClassName("inline-btn-2 r button-dark")[0].click()')
    time.sleep(3)
    driver.find_element(By.ID, "login-username1").send_keys("picaxota2")
    driver.find_element(By.ID, "login-password1").send_keys("123456")

    driver.execute_script("document.getElementsByClassName('inline-btn-2')[2].click()")

    try:
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "tournamentTable")))
        webElement = driver.execute_script('return document.getElementById("tournamentTable").outerHTML')
    except TimeoutException:
        print("A tabela não foi carregada a tempo.")
        # driver.close()
        exit(-1)
 
    print("Jogos baixados com sucesso.")

except Exception as e:
    print(f"Um erro ocorreu ao executar o arquivo.")
    # driver.close()
    exit(-1)

try:
    assert isinstance(webElement, str)
except AssertionError:
    print("A tabela dos jogos não foi baixada corretamente.")
    driver.close()
    exit(-1)

SoupElement = BeautifulSoup(webElement, 'html.parser')

# Obtém todos os jogos disponíveis na tabela
for href in SoupElement.find_all('a', href=True):
    sublink = href["href"].replace("https://www.oddsportal.com", "")
    
    assert isinstance(sublink, str)
    # Esse len(sublink) > 31 serve para remover lixos que estão em alguns hrefs
    # o do sublink != SUBURL indica que o sublink não pode ser igual ao site + SUBURL,
    # que é o site atual
    if len(sublink) > 31 and sublink != SUBURL:
        print(sublink)
        available_games.append(sublink)

for link in available_games:
    df = retrieve_game_parsed_text(driver, "https://www.oddsportal.com" + link)
    print(df)
    print(Arbitrage(
        odd_a = df["Odd_a"].max(),
        odd_b = df["Odd_b"].max(),
        odd_draw = df["Empate"].max(),
        platform= str(),
    ).calculate_bets(verbose=True))

    print('\n')
    print("******************************************************************************")
    print("******************************************************************************")
    print('\n')

driver.close()

Um erro ocorreu ao executar o arquivo.
A tabela dos jogos não foi baixada corretamente.


TypeError: object of type 'NoneType' has no len()